In [2]:
import numpy as np
import pandas as pd
import sys
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


# calculate M for single article

In [3]:
light_dump = []
naruto = False
with open("../data/raw/lightdump/es_wiki.txt", encoding = 'utf8') as file:
    for line in file:
        if (line[0]!= "^") & naruto:
            break
        
        if line.strip() == 'Naruto':
            naruto = True
            continue
            
        if naruto:
            light_dump.append(line)

In [4]:
light_dump.reverse()

In [5]:
light_dump = [x[4:-1].split(' ') for x in light_dump]
light_dump[:5]

[['2004-04-03T13:42:17Z', '0', '1', 'Sabbut'],
 ['2004-04-03T14:09:31Z', '0', '2', 'Sabbut'],
 ['2004-04-11T23:07:42Z', '0', '3', 'Moriel'],
 ['2004-05-23T22:28:21Z', '0', '4', 'Yas'],
 ['2004-07-24T23:14:58Z', '0', '5', 'Sabbut']]

In [6]:
df = pd.DataFrame(data = light_dump, columns = ['time', 'revert', 'index', 'username'])
# num of edits
num_edits = df.username.value_counts()
df['num_edits'] = df.username.apply(lambda x: num_edits[x])

df.head()

,time,revert,index,username,num_edits
0,2004-04-03T13:42:17Z,0,1,Sabbut,18
1,2004-04-03T14:09:31Z,0,2,Sabbut,18
2,2004-04-11T23:07:42Z,0,3,Moriel,1
3,2004-05-23T22:28:21Z,0,4,Yas,1
4,2004-07-24T23:14:58Z,0,5,Sabbut,18


In [7]:
# find revert pairs
revert_pairs = []
revert_users = []
for idx, row in df.iterrows():
    if row['revert']=='1':
        one = row['username']
        org = df.iloc[int(row['index'])-1]
        two = org['username']
        revert_pairs.append((one, two))
        if one not in revert_users:
            revert_users.append(one)
        if two not in revert_users:
            revert_users.append(two)

In [8]:
#find mutual reverts
mutual_rev_users = []
for pair in revert_pairs:
    one = pair[0]
    two = pair[1]
    
    #mutual revert found
    if (two, one) in revert_pairs:
        mutual_rev_users.append(two)
        mutual_rev_users.append(one)

#remove duplicates, calculate num
E = len(list(set(mutual_rev_users)))
E

12

In [9]:
#calculate M
M = 0
num_edits = df.username.value_counts()
revert_pairs = list(set(revert_pairs))
for pair in revert_pairs:
    one = pair[0]
    two = pair[1]
    if num_edits[one] < num_edits[two]:
        N = num_edits[one]
    else:
        N = num_edits[two]

    M += N

M *= E
M

56100

# EDA on light dump

In [ ]:
light_dump = {}
with open("../data/raw/lightdump/en_wiki.txt", encoding = 'utf8') as file:
    for line in file:
        # title
        if line[0]!= "^":
            title = line
            continue
            
        # data
        if line[0] == "^":
            if title in light_dump:
                light_dump[title].append(line)
            else:
                light_dump[title] = [line]

In [ ]:
df = pd.DataFrame(columns = ['time', 'revert', 'index', 'username', 'title'])

counter = 0
for key in light_dump:
    counter += 1
    data = [x[4:-1].split(" ") for x in light_dump[key]]
    chunk = pd.DataFrame(data = data, columns = ['time', 'revert', 'index', 'username'])
    chunk['title'] = key.replace(" ", "").replace("\n", "")
    df = pd.concat([df, chunk])
    if counter == 10000:
        break
    
df.head()

In [ ]:
df.time = pd.to_datetime(df.time)
df.revert = df.revert.astype(int)
df['index'] = df.index.astype(int)
df.head()

In [ ]:
df.revert.value_counts(normalize = True)

In [ ]:
df[df.revert == 1].username.value_counts()[:10]

In [ ]:
df[df.revert == 0].username.value_counts()[:10]

In [ ]:
df.title.value_counts()[-9700:]

In [ ]:
#158 reverts
many = df[df.title == 'Zacarias_Moussaoui']
print(many.revert.sum() / len(many.revert))

many.plot(x = 'time', y = 'revert')

In [ ]:
#little reverts
little = df[df.title == 'Bella_Thorne']
print(little.revert.sum() / len(little.revert))
little.plot(x = 'time', y='revert')

In [ ]:
#little reverts
little = df[df.title == 'Centenary_Medal']
print(little.revert.sum() / len(little.revert))

little.plot(x = 'time', y='revert')

In [ ]:
#little reverts
little = df[df.title == 'Pope_Leo_X']
print(little.revert.sum() / len(little.revert))
little.plot(x = 'time', y='revert')

# participation 3

In [95]:
content_2 = []
with open("../data/testdata/en_wiki_test.txt", encoding = 'utf8') as file:
    
    content_2 = file.readlines()


In [91]:
# wikidump data
revision = wiki_data.find_all("revision")

timestamp = [x.timestamp for x in revision]
comment = [x.comment for x in revision]
username = [x.username for x in revision]

d = {'timestamp': timestamp, 'comment': comment, 'username': username}
df = pd.DataFrame(data = d)

revert = [("revert" in str(x).lower()) for x in comment]
revert = [1 if x== True else 0 for x in revert]

df['revert'] = revert
df.head()

,timestamp,comment,username,revert
0,[2002-02-21T15:11:49Z],[*],[David Parker],0
1,[2002-02-21T15:14:16Z],[remove self-link; some other editing],[Vicki Rosenzweig],0
2,[2002-02-21T16:06:19Z],[*],[David Parker],0
3,[2002-02-24T08:19:24Z],"[Oops! Richard I, not a Robert! And it was mor...",[David Parker],0
4,[2002-02-25T15:51:15Z],[*],[David Parker],0


In [111]:
light_dump = [x for x in content_2 if "^^^" in x]
light_dump[:5]

print(light_dump[0])
light_timestamp = [x[4:24] for x in light_dump]

light_timestamp[:5]

^^^_2009-12-16T02:31:41Z 0 267 128.205.63.225



['2009-12-16T02:31:41Z',
 '2009-12-02T18:08:00Z',
 '2009-12-02T13:55:48Z',
 '2009-11-28T18:33:42Z',
 '2009-11-23T23:14:17Z']

In [131]:
light_timestamp[0]

'2009-12-16T02:31:41Z'

In [135]:
str(timestamp[0]).replace("<timestamp>", "").replace("</timestamp>", "")

'2002-02-21T15:11:49Z'

In [136]:
in_light = [True if str(x).replace("<timestamp>", "").replace("</timestamp>", "") in light_timestamp else False for x in timestamp]
n_light = [1 if x== True else 0 for x in in_light]

df['in_light'] = in_light
df.head()

,timestamp,comment,username,revert,in_light
0,[2002-02-21T15:11:49Z],[*],[David Parker],0,True
1,[2002-02-21T15:14:16Z],[remove self-link; some other editing],[Vicki Rosenzweig],0,True
2,[2002-02-21T16:06:19Z],[*],[David Parker],0,True
3,[2002-02-24T08:19:24Z],"[Oops! Richard I, not a Robert! And it was mor...",[David Parker],0,True
4,[2002-02-25T15:51:15Z],[*],[David Parker],0,True


In [137]:
sum((df.revert == 1) & (df.in_light == 1))

34